In [1]:
from collections import defaultdict
import random
from multiprocessing.pool import ThreadPool
import time
import os


import Tree_simulator_fitting as cts
from Simulate_epidemic_fitting import *
from vector_comparisons import *
from movement_fitting import *


import tempfile
import pyabc

Defining contact structures


In [4]:
results_path = "Looping models/Results/Fitting/LTT/"

run_number = 2

try:
    os.mkdir(os.path.join(dropbox_path, results_path, str(run_number)))
except FileExistsError:
    pass

In [21]:
observed_SS = get_observed_SS()

#print(observed_SS[2], allow_pickle = True)
new_LTT = list(observed_SS[2])
print(type(new_LTT))
observed = [new_LTT, observed_SS[6], observed_SS[7]]


<class 'list'>


In [10]:
def compare_LTT_stats(obs_LTT, coalescent_tree):
    
    sim_LTT_pre = LTT.calculate_LTT_metrics(coalescent_tree.lineages_through_time)
    sim_LTT = normalise(sim_LTT_pre)
    
    LTT_stat_difference = np.linalg.norm(obs_LTT - sim_LTT)
    
    return LTT_stat_difference



In [11]:
def distance(observed, sim):
    
    coalescent_tree = sim[0]
    dist = sim[1]
    ch = sim[2]
    
    observed_LTT = observed[0]
    obs_dist = observed[1]
    obs_ch = observed[2]
    
    a = compare_LTT_stats(observed_LTT, coalescent_tree)
    b = get_jumps(obs_dist, dist)
    c = get_jumps(obs_ch, ch)
    
    distance = (a+b+c)/3
    
    return distance

In [12]:
def simulate_pyabc(parameter):
    result = simulate_epidemic(**parameter)
    return {"data":result}

In [13]:
prior = pyabc.Distribution(a=pyabc.RV("uniform", 0.5, 1), b=pyabc.RV("uniform",0, 0.5), c=pyabc.RV("uniform", 0, 0.15))

abc = pyabc.ABCSMC(simulate_epidemic, prior, distance)

In [22]:
db_path = ("sqlite:///" +
           os.path.join(tempfile.gettempdir(), "test.db"))

abc_id = abc.new(db_path, {"data": observed})

history = abc.run(max_nr_populations=10, minimum_epsilon=0.1)

StatementError: (builtins.ValueError) Object arrays cannot be saved when allow_pickle=False
[SQL: INSERT INTO summary_statistics (sample_id, name, value) VALUES (?, ?, ?)]
[parameters: [{'value': [[0.07647163596358295, 3.289486088152504e-05, 0.20775744412099748, 0.715588994701951, 0.00014903035258703715], 93, 30], 'name': 'data', 'sample_id': 4}]]